<a href="https://colab.research.google.com/github/brunovianna/language-walks/blob/main/language_walks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!mkdir /content/gdrive/sd-playground/
!cd /content/gdrive/sd-playground/
!git clone https://github.com/keras-team/keras-cv.git
!pip install --upgrade keras-cv

In [ ]:
import time
import keras_cv
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt

import math
import numpy as np
from numpy.random import default_rng


model = keras_cv.models.StableDiffusion(img_width=512, img_height=512)
MAX_PROMPT_LENGTH = 77


        

In [ ]:

rng = default_rng()

def plot_images(images):
    plt.figure(figsize=(20, 20))
    for i in range(len(images)):
        ax = plt.subplot(1, len(images), i + 1)
        plt.imshow(images[i])
        plt.axis("off")


def plot_images_2d(images, inverted, zoom):
    if inverted:
      plt.figure(figsize=(zoom*len(images), zoom*len(images[0])))
      for i in range(len(images)):
        for j in range(len(images[i])):
          ax = plt.subplot(len(images[0]), len(images), (j * len(images)) + i +1)
          plt.imshow(images[i][j])
          plt.axis("off")
    else:
      plt.figure(figsize=(zoom*len(image[0]), zoom*len(images)))
      for i in range(len(images)):
        for j in range(len(images[i])):
          ax = plt.subplot(len(images), len(images[0]), (i * len(images)) + j +1)
          plt.imshow(images[i][j])
          plt.axis("off")

def randnsphere(n,radius):
  v = [rng.standard_normal() for i in range(0, n)]
  inv_len = radius / math.sqrt(sum(coord * coord for coord in v))
  return [coord * inv_len for coord in v]

num_points = 10
seed = 10000002
num_steps=25
batch_size = 4
sphere_radius = 5

prompt = "trunk"

tokens = model.tokenizer.encode(prompt)
phrase = tokens + [49407] * (MAX_PROMPT_LENGTH - len(tokens))
phrase = tf.convert_to_tensor([phrase], dtype=tf.int32)
context_base = model.text_encoder.predict_on_batch(
    [phrase, model._get_pos_ids()]
)

img_rows = []
img_rows.append (model.generate_image(context_base,num_steps=num_steps,seed=seed, batch_size=batch_size))

for i in range(num_points):
  context=context_base.copy()
  print (str(i)+" before "+ str(context[0][0][0]))
  s1 = np.array(randnsphere(768,sphere_radius))
  # print (s1)
  around = np.array(context[0][0])+s1
  context[0][0] = around
  print (str(i)+" after "+ str(context[0][0][0]))
  img = model.generate_image(context,num_steps=num_steps,seed=seed, batch_size=batch_size)
  img_rows.append(img)
  # print (img_rows)


plot_images_2d(img_rows,True,3)
